# Understanding nighttime methane signals at the Amazon Tall Tower Observatory (ATTO)

#### Santiago Botía B et al
#### Data management and merging

In [1]:
#import sys
#sys.path.append('/pf/b/b301034/anaconda3/lib/python3.7/site-packages/')
# above lines needed when running from a different environment, not base
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as cdf 
from matplotlib.pyplot import *
import pandas as pd
import seaborn as sns
import scipy
import matplotlib as mpl
# Enabling the warnings: This affects the wind_cluster for loop. in FireSignalAnalysisGeneral I dont have it and it gives different results.
#import warnings 
#warnings.filterwarnings("ignore")
%matplotlib inline

## Importing GHG data, period: 201203-201811
- These dataset was processed by David Walter from raw data to 30-minute averages
- Santiago Botia did a further processing to change to local time and generate a pickle file.

In [2]:
d_clean_mod = pd.read_pickle('./Data/ATTO_GHG_201203_201811_CO2avg')
ch4_30min   = d_clean_mod #.iloc[:,10:15]
ch4_30min   = ch4_30min.assign(LTRound=ch4_30min.index.round('30min'))
thresh      = 8
ch4_30min   = ch4_30min.assign(grad = ch4_30min.CH4_79 - ch4_30min.CH4_4)
ch4_30min   = ch4_30min.assign(GradFlag = ch4_30min['grad']>thresh)
ch4_30min.set_index('LTRound',inplace=True)
ch4_30min.head();

## Importing Black Carbon Data, period: 201401-201512
- These dataset wasprovided by Bruna Holanda, Max Planck for Chemistry in Mainz

In [3]:
bc_data              = pd.read_csv('./Data/maap_30min_2014-2015_STP.csv',delimiter=',')
bc_data['TimeUTC']   = pd.to_datetime(bc_data['TimeUTC'])
bc_data              = bc_data.set_index('TimeUTC')
lt                   = bc_data.index.shift(-4,freq='H')
bc_data['LocalTime'] = lt
bc_data.set_index('LocalTime',inplace=True)
bc_data.head(),bc_data.shape

(                     BC_ugm-3
 LocalTime                    
 2013-12-31 20:00:00  1.930897
 2013-12-31 20:30:00  2.051711
 2013-12-31 21:00:00  1.981338
 2013-12-31 21:30:00  2.003354
 2013-12-31 22:00:00  2.221551, (31121, 1))

## Importing the micromet data: period 201301 - 201812
- These dataset was processed by Marta Sá and Paulo Teixeira in Manaus.
- Note that the datetime is in UTC

In [4]:
inst_data         = pd.read_excel('./Data/INST_aws_precertified_30min_corrected.xlsx',skiprows=1,sheet_name='Plan1')
inst_data.columns = ['Year','doy','Time hhmm','T_81m','T_73m','T_55m','T_40m','T_36m','T_26m','T_12m','T_4m','T_1.5m','T_0.4m','RH_81m','RH_73m','RH_55m','RH_40m','RH_36m','RH_26m','RH_12m','RH_4m','RH_1.5m','RH_0.4m',
                     'SW_in','SW_out','LW_atm','LW_terr','T_LW_atm','T_LW_terr','PAR_in','PAR_out','UV','NetRad','Patm_81m','Rainfall', 'WSp_73m','WDir_73m','WSp_65m','WDir_65m','Bat_Aws','WSp_50m','WDir_50m','WSp_42m',
                     'WDir_42m','WSp_26m','WDir_26m','WSp_19m','WDir_19m','bat_soil','LW_atm_calc','LW_terr_calc','SD', 'SD.1', 'SD.2', 'SD.3', 'SD.4', 'SD.5', 'SD.6', 'SD.7', 'SD.8', 'SD.9', 'SD.10', 'SD.11', 'SD.12',
                     'SD.13', 'SD.14', 'SD.15', 'SD.16', 'SD.17', 'SD.18', 'SD.19', 'SD.20', 'SD.21', 'SD.22','SD.23', 'SD.24', 'SD.25', 'SD.26', 'SD.27', 'SD.28', 'SD.29', 'SD.30', 'SD.31', 'SD.32', 'SD.33', 'SD.34', 
                     'SD.35', 'SD.36', 'SD.37', 'SD.38', 'SD.39', 'SD.40', 'SD.41', 'SD.42', 'SD.43', 'SD.44','SD.45', 'SD.46','SD.47']
dates             = pd.date_range('2013-01-01 00:30','2019-01-01 00:00',freq='0.5H')
inst_data         = inst_data.assign(datetime_UTC = dates)
inst_data.set_index('datetime_UTC',inplace=True)
inst_data.head()

,Year,doy,Time hhmm,T_81m,T_73m,T_55m,T_40m,T_36m,T_26m,T_12m,...,SD.38,SD.39,SD.40,SD.41,SD.42,SD.43,SD.44,SD.45,SD.46,SD.47
datetime_UTC,,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:30:00,2013,1,30,23.745000,23.887000,24.027667,NaN,23.765000,23.038667,23.499667,...,21.991351,0.199234,21.362121,0.097781,29.876451,0.072462,39.902539,NaN,0.439403,0.297708
2013-01-01 01:00:00,2013,1,100,23.743333,24.007667,24.003333,NaN,23.654333,23.102000,23.410000,...,9.230696,0.412777,13.256965,0.112999,19.296953,0.091028,73.136787,NaN,1.057253,0.359579
2013-01-01 01:30:00,2013,1,130,23.696000,23.844333,23.781667,NaN,23.608333,23.136333,23.406667,...,8.636730,0.419433,9.684684,0.127305,53.146771,0.080370,82.629460,NaN,0.980883,0.548035
2013-01-01 02:00:00,2013,1,200,23.720333,23.893333,23.741667,NaN,23.467333,23.042000,23.375333,...,9.493131,0.363381,7.232112,0.138608,66.371516,0.062944,98.673181,NaN,0.706343,0.599715
2013-01-01 02:30:00,2013,1,230,23.497333,23.774667,23.698333,NaN,23.265333,22.893000,23.317333,...,14.682110,0.254188,11.441254,0.120444,59.639424,0.077055,66.442863,NaN,0.380992,0.407596


### Virtual potential temperature and potential temperature calculation
#### These variable are calcuted from the temperature profile as follows
- values of ql must be in kg/kg

$\theta_v = \theta(1 + 0.61q - q_l)$ 

$q = RH_{frac} * qs$

$qs = \frac{(0.622*es)}{(pres/10)}$

$es = e_0*(2.7182)^{\frac{(b*(tk-t1)}{tk-t2}} $

In [5]:
def pot_temp_v(temp,rh,pot_temp,pres):
    # function for virtual potential temperature
    Rcp   = 0.286                                   #r/cp = 287/1004 [Joule K-1 kg-1/Joule K-1 kg-1]
    e0    = 611                                     #[Pa]
    b     = 17.2694                                 #dimensionless
    t1    = 273.16                                  #[K]
    t2    = 35.86                                   #[K]
    temp  = temp + 273.15 
    es    = e0*(2.718281**((b*(temp-t1))/(temp-t2)))    #water vapor pressure
    qs    = (0.622*es)/(pres/10)                        #saturated specific humidity
    rh1   = rh/100                                       #relative humidity
    q     = (rh1*qs) 
    ql    = q - qs
    
    ql[ql<0] = 0
    pot_temp_virt = pot_temp*(1+0.61*(q-ql)/1000)
    
    return pot_temp_virt,es,qs,q,ql

In [6]:
# Calculating the potential temperature at all heights and based on this the cirtual pot temp
theta1  = inst_data['T_81m'] + 273.15 + (9.8 / 1004) * 81
theta2  = inst_data['T_73m'] + 273.15 + (9.8 / 1004) * 73
theta3  = inst_data['T_55m'] + 273.15 + (9.8 / 1004) * 55
theta4  = inst_data['T_40m'] + 273.15 + (9.8 / 1004) * 40
theta5  = inst_data['T_36m'] + 273.15 + (9.8 / 1004) * 36
theta6  = inst_data['T_26m'] + 273.15 + (9.8 / 1004) * 26
theta7  = inst_data['T_12m'] + 273.15 + (9.8 / 1004) * 12
theta8  = inst_data['T_4m']  + 273.15 + (9.8 / 1004) * 4
theta9  = inst_data['T_1.5m'] + 273.15 + (9.8 / 1004) * 1.5
theta10 = inst_data['T_0.4m'] + 273.15 + (9.8 / 1004) * 0.4

inst_data = inst_data.assign(theta = theta1)
inst_data = inst_data.assign(theta73 = theta2)
inst_data = inst_data.assign(theta55 = theta3)
inst_data = inst_data.assign(theta40 = theta4)
inst_data = inst_data.assign(theta36 = theta5)
inst_data = inst_data.assign(theta26 = theta6)
inst_data = inst_data.assign(theta12 = theta7)
inst_data = inst_data.assign(theta4 = theta8)
inst_data = inst_data.assign(theta1_5 = theta9)
inst_data = inst_data.assign(theta0_4 = theta10)

inst_data = inst_data.assign(PotVirt_81m = pot_temp_v(inst_data.T_81m,inst_data.RH_81m,inst_data.theta,inst_data.Patm_81m)[0])
inst_data = inst_data.assign(PotVirt_73m = pot_temp_v(inst_data.T_73m,inst_data.RH_73m,inst_data['theta73'],inst_data.Patm_81m)[0])
inst_data = inst_data.assign(PotVirt_55m = pot_temp_v(inst_data.T_55m,inst_data.RH_55m,inst_data['theta55'],inst_data.Patm_81m)[0])
inst_data = inst_data.assign(PotVirt_40m = pot_temp_v(inst_data.T_40m,inst_data.RH_40m,inst_data['theta40'],inst_data.Patm_81m)[0])
inst_data = inst_data.assign(PotVirt_36m = pot_temp_v(inst_data.T_36m,inst_data.RH_36m,inst_data['theta36'],inst_data.Patm_81m)[0])
inst_data = inst_data.assign(PotVirt_26m = pot_temp_v(inst_data.T_26m,inst_data.RH_26m,inst_data['theta26'],inst_data.Patm_81m)[0])
inst_data = inst_data.assign(PotVirt_12m = pot_temp_v(inst_data.T_12m,inst_data.RH_12m,inst_data['theta12'],inst_data.Patm_81m)[0])
inst_data = inst_data.assign(PotVirt_4m = pot_temp_v(inst_data.T_4m,inst_data.RH_4m,inst_data['theta4'],inst_data.Patm_81m)[0])
inst_data = inst_data.assign(PotVirt_1_5m = pot_temp_v(inst_data['T_1.5m'],inst_data['RH_1.5m'],inst_data['theta1_5'],inst_data.Patm_81m)[0])
inst_data = inst_data.assign(PotVirt_0_4m = pot_temp_v(inst_data['T_0.4m'],inst_data['RH_0.4m'],inst_data['theta0_4'],inst_data.Patm_81m)[0])

In [7]:
# exporting to ./Data
#inst_data.to_pickle('./Data/df_inst_data')

## Importing the eddy covariance data: period 201301 - 201812
- These dataset was processed by Marta Sá and Paulo Teixeira in Manaus.
- Note that the datetime is in UTC

In [8]:
fluxes_inst         = pd.read_excel('./Data/INST_fluxes_precertified_30min.xlsx',skiprows=1,sheet_name='Plan1')
fluxes_inst.columns = ['Year','doy','DecTime','FluxTsonic[Watt/m2]','CO2_flux[umol/m2/s]','Flux_Op-H2O[Watt/m2]','Qf_f_tsonic','Qf_F_Op-CO2','Qf_F_Op-H2O','Flux_Tau[kg]/m2/s','Mean_Windsp[m/s]',
                       'Mean_Tsonic[C]','Mean_Op-CO2[umol/mol]','Mean_Op-H2O[gram/m3]','Mean_APress[Pa]','U-star[m/s]','Z-over-L','Wind-Direc-corrg[deg]','80PercFlux[meter]']

dates               = pd.date_range('2013-01-01 00:30','2019-01-01 00:00',freq='0.5H')
fluxes_inst         = fluxes_inst.assign(datetime_UTC = dates)
fluxes_inst.set_index('datetime_UTC',inplace=True)
fluxes_inst.head()

,Year,doy,DecTime,FluxTsonic[Watt/m2],CO2_flux[umol/m2/s],Flux_Op-H2O[Watt/m2],Qf_f_tsonic,Qf_F_Op-CO2,Qf_F_Op-H2O,Flux_Tau[kg]/m2/s,Mean_Windsp[m/s],Mean_Tsonic[C],Mean_Op-CO2[umol/mol],Mean_Op-H2O[gram/m3],Mean_APress[Pa],U-star[m/s],Z-over-L,Wind-Direc-corrg[deg],80PercFlux[meter]
datetime_UTC,,,,,,,,,,,,,,,,,,,
2013-01-01 00:30:00,13,1,0.5,-5.31690,-0.727630,-4.45860,4.0,7.0,1.0,0.012730,1.29170,29.581,390.84,23.096,98600.0,0.106470,3.13570,218.410,55238.0
2013-01-01 01:00:00,13,1,1.0,-1.52080,-0.029413,-1.79500,1.0,101.0,1.0,0.002553,0.81590,29.789,391.24,23.210,98600.0,0.047695,9.90440,232.800,94238.0
2013-01-01 01:30:00,13,1,1.5,-2.18930,-0.531530,0.68189,1.0,4.0,1.0,0.015003,1.01760,29.640,390.98,23.224,98620.0,0.115590,0.91915,259.750,25504.0
2013-01-01 02:00:00,13,1,2.0,0.67955,0.025135,0.42648,1.0,1.0,1.0,0.014039,0.82103,29.586,391.06,23.155,98685.0,0.111770,-0.37557,281.710,2918.0
2013-01-01 02:30:00,13,1,2.5,-2.05040,0.711520,0.50580,1.0,4.0,1.0,0.007760,0.20258,29.716,391.51,23.149,98674.0,0.083121,2.50980,43.766,10654.0


## Importing the soil moisture and soil temperature data: period 201301 - 201812
- These dataset was processed by Marta Sá and Paulo Teixeira in Manaus.
- Note that the datetime is in UTC

In [9]:
soilmois         = pd.read_excel('./Data/INST_soil_precertified_30min.xlsx',skiprows=1,sheet_name='Plan1',dtype='object')
soilmois.columns = ['Year','Doy','Time','bat','STemp_10cm','STemp_20cm','STemp_40cm','Smois_10cm',
                    'Smois_20cm','Smois_30cm','Smois_40cm','Smois_60cm','Smois_100cm','Flx_s[W/m2]']
dates            = pd.date_range('2013-01-01 00:30','2019-01-01 00:00',freq='0.5H')
soilmois         = soilmois.assign(datetime_UTC = dates)  
soilmois.set_index('datetime_UTC',inplace=True)
soilmois = soilmois.astype(np.float64)

#### Here I calculated the WFPS, according to Andreae et al., 2015:
- depths bulk density profile  = 0.1, 0.2,  0.3,  0.5 , 1,   1.5, 2
- bulk density profile [g cm$^{-3}$]        = 0.5, 0.83, 0.98, 1.1 , 1.1, 1,   1.1 

In [10]:
def get_wfps(bd,pd,soilmoist):
    #for each depth receives bulk density, particle density and soil moisture
    grav_smois     = soilmoist / bd
    total_porosity = 1 - (bd / pd)
    wfps           = ((grav_smois*bd)/total_porosity) * 100
    
    return wfps

In [11]:
# Particle density is assumed to be 2.86 based on Schjønning et al., 2017, see paper references
soilmois = soilmois.assign(WFPS_10cm = get_wfps(0.5,2.86,soilmois['Smois_10cm']))
soilmois = soilmois.assign(WFPS_20cm = get_wfps(0.83,2.86,soilmois['Smois_20cm']))
soilmois = soilmois.assign(WFPS_30cm = get_wfps(0.98,2.86,soilmois['Smois_30cm']))
soilmois = soilmois.assign(WFPS_40cm = get_wfps(1,2.86,soilmois['Smois_40cm']))
soilmois = soilmois.assign(WFPS_60cm = get_wfps(1.1,2.86,soilmois['Smois_60cm']))
soilmois = soilmois.assign(WFPS_100cm = get_wfps(1.1,2.86,soilmois['Smois_100cm']))

## Data arrangement for boxplot: Fig 3.

In [12]:
#selecting per inlet all the data and organizing properly to plot a boxplot
from pandas import Series
from pandas import DataFrame
from pandas import concat
import matplotlib.ticker as mticker
import matplotlib.dates as mdates

dfs = [ch4_30min]
for num,data in enumerate(dfs):
    df      = data
    species = ['CH4_79','CH4_53', 'CH4_38', 'CH4_24', 'CH4_4']
    for i,j in enumerate(species):
        boxprops       = dict(linestyle='-', linewidth=2)
        medianprops    = dict(linestyle='-', linewidth=2)
        one_year       = df[j]
        groups         = one_year['2013-06':'2018-12'].groupby(pd.Grouper(freq='M'))
        months         = concat([DataFrame(x[1].values) for x in groups], axis=1)
        months         = DataFrame(months)
        months.columns = pd.date_range(start='2013-06',end='2018-12',freq='M') # Monthly plot
        months.rename(columns=lambda t: t.strftime('%Y-%m'),inplace=True) # Monthly plot
        
        if i == 0: months79 = months
        if i == 1: months53 = months
        if i == 2: months38 = months
        if i == 3: months24 = months
        if i == 4: months4  = months
            
months79 = months79.assign(h = '79m')
months53 = months53.assign(h = '53m')
months38 = months38.assign(h = '38m')
months24 = months24.assign(h = '24m')
months4  = months4.assign(h = '4m')

months79 = pd.melt(months79,id_vars=['h'])
months53 = pd.melt(months53,id_vars=['h'])
months38 = pd.melt(months38,id_vars=['h'])
months24 = pd.melt(months24,id_vars=['h'])
months4  = pd.melt(months4,id_vars=['h'])

In [13]:
# Exporting the data for plotting in the correct format.
dfs = [months79,months53,months38,months24,months4]
labs = ['79m','53m','38m','24m','4m']

for j,i in enumerate(dfs):
    i.columns = ['Inlet Height', 'Month', 'CH4[ppb]']
    #i.to_pickle('./Data/ATTO_CH4_%s'%(labs[j]))

## Merging concentrations, EC data, soil data and micromet data
- To merge these dataframes, the index of each one has to be on the same timescale.

In [29]:
# Shifted to UTC for merging purposes, merging with inst_data and fluxes_inst
ch4_30minUTC = ch4_30min.assign(UTCRound=ch4_30min.index.shift(4,freq='H'))
ch4_30minUTC.set_index('UTCRound',inplace=True)
ch4_30minUTC.head()

,CO2_avg_79,CO2_avg_53,CO2_avg_38,CO2_avg_24,CO2_avg_4,CO_79,CO_53,CO_38,CO_24,CO_4,CH4_79,CH4_53,CH4_38,CH4_24,CH4_4,grad,GradFlag
UTCRound,,,,,,,,,,,,,,,,,
2012-03-07 00:30:00,396.435,396.110,399.82,420.835,425.660,115.15,117.60,123.04,124.25,125.12,1839.65,1840.56,1841.10,1841.07,1841.22,-1.57,False
2012-03-07 01:00:00,394.725,395.110,400.11,419.660,425.500,112.00,113.77,115.29,126.59,124.30,1839.05,1839.98,1840.43,1841.53,1842.18,-3.13,False
2012-03-07 01:30:00,394.485,393.575,398.27,422.125,425.915,113.42,115.91,117.50,122.92,121.02,1839.52,1841.29,1840.31,1841.67,1842.13,-2.61,False
2012-03-07 02:00:00,396.455,394.385,398.29,423.610,430.710,117.44,115.81,117.95,123.24,127.89,1841.13,1841.85,1840.89,1842.17,1843.04,-1.91,False
2012-03-07 02:30:00,399.575,397.465,399.37,422.125,433.705,112.63,119.11,117.72,126.17,125.21,1841.74,1842.83,1841.99,1842.62,1843.40,-1.66,False


In [40]:
ch4_30minUTC.shape,inst_data.shape,fluxes_inst.shape

((118079, 17), (105168, 119), (105168, 19))

In [32]:
# Merging data for same times. 
#df_full  = pd.concat([ch4_30minUTC,inst_data,fluxes_inst,soilmois],axis=1,join='inner') # el de antes que genera inconsistencia en netrad/atmplot
df_full  = pd.concat([ch4_30minUTC['2013-06':],inst_data,fluxes_inst,soilmois],axis=1,join='inner')
# Be aware of the dates of merging. 
#Changing back to local time for analyses purposes.
df_full  = df_full.assign(LTime=df_full.index.shift(-4,freq='H'))
df_full.set_index('LTime',inplace=True)
df_full  = df_full.assign(Hour = df_full.index.hour)
df_full.head()

,CO2_avg_79,CO2_avg_53,CO2_avg_38,CO2_avg_24,CO2_avg_4,CO_79,CO_53,CO_38,CO_24,CO_4,...,Smois_60cm,Smois_100cm,Flx_s[W/m2],WFPS_10cm,WFPS_20cm,WFPS_30cm,WFPS_40cm,WFPS_60cm,WFPS_100cm,Hour
LTime,,,,,,,,,,,,,,,,,,,,,
2013-05-31 20:00:00,396.745,398.870,399.915,401.955,448.900,94.79,91.45,94.24,93.95,103.50,...,0.376,0.341,-0.028333,27.388136,27.613793,41.682979,48.589247,61.1,55.4125,20
2013-05-31 20:30:00,397.420,399.915,401.360,403.250,433.060,97.91,94.54,93.01,94.43,102.58,...,0.376,0.341,-0.397333,27.388136,27.613793,41.682979,48.589247,61.1,55.4125,20
2013-05-31 21:00:00,398.295,400.870,402.565,405.495,428.985,91.62,98.10,92.65,95.74,101.44,...,0.376,0.341,-0.690000,27.388136,27.613793,41.682979,48.589247,61.1,55.4125,21
2013-05-31 21:30:00,399.315,401.850,403.755,407.230,439.495,100.66,97.93,95.32,97.26,99.63,...,0.376,0.341,-0.956000,27.388136,27.472906,41.682979,48.589247,61.1,55.4125,21
2013-05-31 22:00:00,398.760,402.150,404.210,406.825,429.260,97.45,100.09,100.30,99.79,101.08,...,0.376,0.341,-1.182667,27.388136,27.472906,41.682979,48.589247,61.1,55.4125,22


In [42]:
df_full.shape

(96432, 176)

In [33]:
# To export
#df_full.to_pickle('./Data/ATTO_GHG_Inst_flux_soil_30min')
#df_full.to_csv('./Data/ATTO_GHG_Inst_flux_soil_30min.csv')

##  Merging the data for the 1 minute Analysis
### Figure 10
- This merging consists of high-frequency data of
    - u, v and w averaged at 1min. The averaging to 1 min from 10 Hz data was performed in Rstudio. The original code was given by Michel Stefanello and modified by Santiago Botia. The code is available in this directory. The original data is available with Marta Sá.
    - Atmospheric CH$_4$ mixing ratios at 15-minute resolution.
    - meteorological variables at 1 min resolution (available with Marta Sá)
    - The "Sbotia_Et_al_1minMerging.R" was run in Apr-2019 in my local computer. It generated the 1min resoltuion file that I import here: "ATTO_81m_2014_03-09_u_v_w.1min.txt". Strangely running this script on Mistral misses the first hour.. Dont know why. Using the same script locally, it works.
    
- Note that 
    - ATTO_81m_2014_03-09_u_v_w.1min.txt is the correct one
    - ATTO_81m_2014_03-09_u_v_w.1min_3.txt misses the first hour.

In [14]:
# These files are in pickle format. 
# They were generated by SBotia based on data send by David Walter
atto_79_full    = pd.read_pickle('./Data/ATTO_79m_15min_201401-201807_pckl')
atto_4_full     = pd.read_pickle('./Data/ATTO_4m_15min_201401-201807_pckl')
minFile         = pd.read_table('./Data/ATTO_81m_2014_03-09_u_v_w.1min.txt',header=None)
#minFile         = pd.read_table('./Data/ATTO_81m_2014_03-09_u_v_w.1min_2.txt',header=None)
minFile3         = pd.read_table('./Data/ATTO_81m_2014_03-09_u_v_w.1min_3.txt',header=None)

minFile.columns = ["datetime","angolo","thetamed", "sigma_theta","mean_u","mean_v","mean_w", 
                   "mean_T","sigma_u","sigma_v", "sigma_w","sigma_T", "uw","vw","uT","vT","wT"]
all_h           = pd.read_pickle('./Data/meteovars_1min_pot_rnet')

In [15]:
# high frequency data averaged at 1min, shifting index to local time
minFile         = minFile.set_index('datetime')
minFile.index   = pd.to_datetime(minFile.index)
lt              = minFile.index.shift(-4,freq='H')
minFile['LocalTime'] = lt
minFile.set_index('LocalTime',inplace=True)
minFile         = minFile.assign(EnhanFlag = np.nan)

# Adjusting concentration data from 15 min to 1 min by assuming the same concentration for 15min
atto_4_full.reset_index(inplace=True)
atto_79_full.reset_index(inplace=True)
# Rounding to closest 15min
atto_4_full      = atto_4_full.assign(LTRound = atto_4_full['LocalTime'].dt.round('15min'))
atto_79_full     = atto_79_full.assign(LTRound = atto_79_full['LocalTime'].dt.round('15min'))
atto_4_full.set_index('LTRound',inplace=True)
atto_79_full.set_index('LTRound',inplace=True)
# Changing freq to 1min and filling in gaps
atto_79_full_2014 = atto_79_full['2014'].resample('1T').mean()
atto_4_full_2014  = atto_4_full['2014'].resample('1T').mean()
atto_4_full_2014.fillna(method='ffill',inplace=True)
atto_79_full_2014.fillna(method='ffill',inplace=True)

In [16]:
# Getting the concentration gradient and merging all dataframes based on the MinFile dates
df_grad         = atto_79_full_2014['2014-02':'2014-10'] - atto_4_full_2014['2014-02':'2014-10']
df_grad.columns = ['inlet','CO_grad','CO2_grad1','CH4_grad','CO2_grad2']
df_grad         = df_grad.assign(CH4_79 = atto_79_full_2014.loc['2014-02':'2014-10','D9CH4'] )
df_grad         = df_grad.assign(CH4_4  = atto_4_full_2014.loc['2014-02':'2014-10','D9CH4'] )
df_full         = pd.concat([minFile,df_grad,all_h],axis=1,join='inner')

# Deleteing uncessary columns
del df_full['angolo']
del df_full['thetamed']
del df_full['sigma_theta']
del df_full['inlet']

In [17]:
# Adding wind direction clusters to full dataframe as well as the mean wind speed at 81m
df_full = df_full.assign(mean_wind81 = np.sqrt(df_full['mean_u']**2+df_full['mean_v']**2))
df_full = df_full.assign(Ustar = (df_full.uw**2 + df_full.vw**2)**(1/4)) #(u'w'^2 + v'w'^2) ^(1/4)
df_full = df_full.assign(hour = df_full.index.hour)
direcs  = ['NNE','ENE','ESE','SSE','SSW','WSW','WNW','NNW']
df_full = df_full.assign(cluster = np.nan)

for i,j in enumerate(np.arange(0,360,45),start=0):
    #print(i,j)
    df_full['cluster'].loc[(df_full['WDir_73m'] >= j) & (df_full['WDir_73m'] < j+45)] = direcs[i]

# adding wind speed classes to full dataframe
#bins    = [np.arange(0,6.5,0.5),np.arange(7,12,1),np.arange(12.5,20,2.5)]
#bins    = [np.arange(0,4,0.5),np.arange(4,8,0.2),np.arange(8,15,1)]
bins    = [np.arange(0,6,0.4),np.arange(6,15,1)] # final bin configuration used in paper
bins    = np.concatenate(bins).ravel()
# Adding wind speed class bins, the gradient flag, the potential temperature gradient between 81 and 4 and wind speed gradients
df_full = df_full.assign(WindClass = pd.cut(df_full['mean_wind81'],bins,right=True))
df_full = df_full.assign(EnhanGrad = df_full['CH4_grad']>8)
#df_full = df_full.assign(PotTempGrad81_4 = df_full['theta']-df_full['theta4'])
#df_full = df_full.assign(WspGrad73_26 = df_full['WSp_73m']-df_full['WSp_26m'])

def mid(interval):
    return interval.mid

df_full = df_full.assign(WindClassMid = df_full.WindClass.apply(mid))

/pf/b/b301034/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [67]:
# No need to limit to nighttime, this is done when generating the plot
#df_full.to_pickle('./Data/df_GHG_uvw_1min_merged')

## Merging the BC data with the CO and CH4 mixing ratios
### Figures 12 and 13

In [22]:
# Changing the fluxes_inst to local time, so it can be merged with the BC data and the CH4 and CO data
fluxes_instLT            = fluxes_inst
lt                       = fluxes_instLT.index.shift(-4,freq='H')
fluxes_instLT['LocalTime'] = lt
fluxes_instLT.set_index('LocalTime',inplace=True)
fluxes_instLT.head()

,Year,doy,DecTime,FluxTsonic[Watt/m2],CO2_flux[umol/m2/s],Flux_Op-H2O[Watt/m2],Qf_f_tsonic,Qf_F_Op-CO2,Qf_F_Op-H2O,Flux_Tau[kg]/m2/s,Mean_Windsp[m/s],Mean_Tsonic[C],Mean_Op-CO2[umol/mol],Mean_Op-H2O[gram/m3],Mean_APress[Pa],U-star[m/s],Z-over-L,Wind-Direc-corrg[deg],80PercFlux[meter]
LocalTime,,,,,,,,,,,,,,,,,,,
2012-12-31 20:30:00,13,1,0.5,-5.31690,-0.727630,-4.45860,4.0,7.0,1.0,0.012730,1.29170,29.581,390.84,23.096,98600.0,0.106470,3.13570,218.410,55238.0
2012-12-31 21:00:00,13,1,1.0,-1.52080,-0.029413,-1.79500,1.0,101.0,1.0,0.002553,0.81590,29.789,391.24,23.210,98600.0,0.047695,9.90440,232.800,94238.0
2012-12-31 21:30:00,13,1,1.5,-2.18930,-0.531530,0.68189,1.0,4.0,1.0,0.015003,1.01760,29.640,390.98,23.224,98620.0,0.115590,0.91915,259.750,25504.0
2012-12-31 22:00:00,13,1,2.0,0.67955,0.025135,0.42648,1.0,1.0,1.0,0.014039,0.82103,29.586,391.06,23.155,98685.0,0.111770,-0.37557,281.710,2918.0
2012-12-31 22:30:00,13,1,2.5,-2.05040,0.711520,0.50580,1.0,4.0,1.0,0.007760,0.20258,29.716,391.51,23.149,98674.0,0.083121,2.50980,43.766,10654.0


In [23]:
# Selecting the CO and CH4 data from the dataframe imported above
ch4_co_30min  = pd.DataFrame()
ch4           = ch4_30min.iloc[:,10:15]
co            = ch4_30min.iloc[:,5:10]
ch4_co_30min  = pd.concat([ch4,co], axis=1)
#print(ch4_co_30min.head())

# Adding the 8ppb threshold to the new df
thresh        = 8
ch4_co_30min  = ch4_co_30min.assign(grad = ch4.CH4_79 - ch4.CH4_4)
# dropping nan
print(ch4_co_30min.shape, 'with nan')
ch4_co_30min.dropna(inplace=True)
print(ch4_co_30min.shape, 'after dropping nan')
# setting threshold flag
ch4_co_30min = ch4_co_30min.assign(GradFlag = ch4_co_30min['grad']>thresh)

(118079, 11) with nan
(76603, 11) after dropping nan


In [24]:
# Including BC
df_full_bc    = pd.concat([bc_data,ch4_co_30min,fluxes_instLT['Wind-Direc-corrg[deg]']],axis=1,join='inner')
# Excluding BC
df_full_nobc  = pd.concat([ch4_co_30min,fluxes_instLT['Wind-Direc-corrg[deg]']],axis=1,join='inner')

In [25]:
df_full_bc.shape,df_full_nobc.shape,fluxes_inst['Wind-Direc-corrg[deg]'].shape,ch4_co_30min.shape

((25176, 14), (73794, 13), (105168,), (76603, 12))

In [26]:
# Setting up the wind clusters
direcs  = ['NNE','ENE','ESE','SSE','SSW','WSW','WNW','NNW']
df_full_bc = df_full_bc.assign(wind_cluster = np.nan)
df_full_nobc = df_full_nobc.assign(wind_cluster = np.nan)

for i,j in enumerate(np.arange(0,360,45),start=0):
    print(i,j)
    df_full_bc['wind_cluster'].loc[(df_full_bc['Wind-Direc-corrg[deg]'] >= j) & (df_full_bc['Wind-Direc-corrg[deg]'] < j+45)] = direcs[i]
    df_full_nobc['wind_cluster'].loc[(df_full_nobc['Wind-Direc-corrg[deg]'] >= j) & (df_full_nobc['Wind-Direc-corrg[deg]'] < j+45)] = direcs[i]

0 0
1 45
2 90


/pf/b/b301034/anaconda3/envs/base_bckup/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


3 135
4 180
5 225
6 270
7 315


In [27]:
# Exporting pickle files
df_full_bc.to_pickle('./Data/df_ch4_co_bc_ratios')
df_full_nobc.to_pickle('./Data/df_ch4_co_ratios')